# api_dev_correlation_function_development
* Author: Jea Kwon
* Description: developing rolling window correlation function in avatar

In [1]:
import datetime
import avatarpy
today = datetime.datetime.today().strftime('%Y-%m-%d')
print('Date: ', today, '\nVersion: ', avatarpy.__version__)

Date:  2021-05-06 
Version:  0.0.4


In [2]:
from avatarpy import Avatar, dataset
import os
import glob
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly.express as px
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
cf.go_offline(connected=True)

In [3]:
csv_path = dataset['freely_moving']

In [4]:
ava = Avatar(csv_path)

# Example) limb-wise velocity correlation

In [5]:
corr = ava.velocity.corr()
corr

,nose,neck,anus,chest,rfoot,lfoot,rhand,lhand,tip
nose,1.000000,0.846679,0.337655,0.666609,0.202680,0.208653,0.490393,0.503752,0.230852
neck,0.846679,1.000000,0.469334,0.807822,0.279971,0.286010,0.565957,0.562241,0.323338
anus,0.337655,0.469334,1.000000,0.744020,0.627957,0.611615,0.426210,0.410555,0.560513
chest,0.666609,0.807822,0.744020,1.000000,0.500490,0.501775,0.586647,0.564098,0.421291
rfoot,0.202680,0.279971,0.627957,0.500490,1.000000,0.246747,0.260187,0.282862,0.385252
lfoot,0.208653,0.286010,0.611615,0.501775,0.246747,1.000000,0.301125,0.264743,0.356357
rhand,0.490393,0.565957,0.426210,0.586647,0.260187,0.301125,1.000000,0.352924,0.258368
lhand,0.503752,0.562241,0.410555,0.564098,0.282862,0.264743,0.352924,1.000000,0.288193
tip,0.230852,0.323338,0.560513,0.421291,0.385252,0.356357,0.258368,0.288193,1.000000


* Output: exact square,  
to remove duplicate, apply triangular mask

In [6]:
mask = np.tril(np.ones(corr.shape).astype(np.bool))
corr.mask(mask)

,nose,neck,anus,chest,rfoot,lfoot,rhand,lhand,tip
nose,NaN,0.846679,0.337655,0.666609,0.202680,0.208653,0.490393,0.503752,0.230852
neck,NaN,NaN,0.469334,0.807822,0.279971,0.286010,0.565957,0.562241,0.323338
anus,NaN,NaN,NaN,0.744020,0.627957,0.611615,0.426210,0.410555,0.560513
chest,NaN,NaN,NaN,NaN,0.500490,0.501775,0.586647,0.564098,0.421291
rfoot,NaN,NaN,NaN,NaN,NaN,0.246747,0.260187,0.282862,0.385252
lfoot,NaN,NaN,NaN,NaN,NaN,NaN,0.301125,0.264743,0.356357
rhand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.352924,0.258368
lhand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288193
tip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# total unique inter-node correlations
corr.mask(mask).count().sum()

36

In [8]:
# flatten 2d-arr
corr_flatten = corr.mask(mask).stack().reset_index()
corr_flatten

,level_0,level_1,0
0,nose,neck,0.846679
1,nose,anus,0.337655
2,nose,chest,0.666609
3,nose,rfoot,0.202680
4,nose,lfoot,0.208653
5,nose,rhand,0.490393
6,nose,lhand,0.503752
7,nose,tip,0.230852
8,neck,anus,0.469334
9,neck,chest,0.807822


In [9]:
labels = pd.Index(['-'.join(x) for x in zip( corr_flatten['level_0'], corr_flatten['level_1'])])
corrs = corr_flatten.set_index(labels)[0]

In [10]:
corrs

nose-neck      0.846679
nose-anus      0.337655
nose-chest     0.666609
nose-rfoot     0.202680
nose-lfoot     0.208653
nose-rhand     0.490393
nose-lhand     0.503752
nose-tip       0.230852
neck-anus      0.469334
neck-chest     0.807822
neck-rfoot     0.279971
neck-lfoot     0.286010
neck-rhand     0.565957
neck-lhand     0.562241
neck-tip       0.323338
anus-chest     0.744020
anus-rfoot     0.627957
anus-lfoot     0.611615
anus-rhand     0.426210
anus-lhand     0.410555
anus-tip       0.560513
chest-rfoot    0.500490
chest-lfoot    0.501775
chest-rhand    0.586647
chest-lhand    0.564098
chest-tip      0.421291
rfoot-lfoot    0.246747
rfoot-rhand    0.260187
rfoot-lhand    0.282862
rfoot-tip      0.385252
lfoot-rhand    0.301125
lfoot-lhand    0.264743
lfoot-tip      0.356357
rhand-lhand    0.352924
rhand-tip      0.258368
lhand-tip      0.288193
Name: 0, dtype: float64

## Applying Rolling Window for Correlation

Note that there is kwarg applied to rolling window, `center=True`.  
The calculated correlation values will be appear at center of window.

In [11]:
window = 20
rolling_corr = ava.velocity[['rfoot', 'lfoot', 'lhand', 'rhand']].rolling(window, center=True).corr()
rolling_corr.dropna()

rfoot     lfoot     lhand     rhand
0.55  rfoot  1.000000  0.737912  0.384960 -0.323462
      lfoot  0.737912  1.000000  0.420771  0.259220
      lhand  0.384960  0.420771  1.000000 -0.230745
      rhand -0.323462  0.259220 -0.230745  1.000000
0.60  rfoot  1.000000  0.673841  0.269956 -0.304451
...               ...       ...       ...       ...
99.45 rhand -0.081409  0.600330  0.043031  1.000000
99.50 rfoot  1.000000  0.312290  0.174261 -0.086313
      lfoot  0.312290  1.000000  0.056661  0.639690
      lhand  0.174261  0.056661  1.000000  0.058932
      rhand -0.086313  0.639690  0.058932  1.000000

[7920 rows x 4 columns]

## Function for flattening correlation matrix

In [24]:
def corr_flatten(df):
    mask = np.tril(np.ones(df.shape).astype(np.bool))
    s = df.mask(mask).stack()
    s.index = s.index.map(lambda x: '_'.join(x[-2:]))
    return s

In [25]:
corr_flatten(corr)

nose_neck      0.846679
nose_anus      0.337655
nose_chest     0.666609
nose_rfoot     0.202680
nose_lfoot     0.208653
nose_rhand     0.490393
nose_lhand     0.503752
nose_tip       0.230852
neck_anus      0.469334
neck_chest     0.807822
neck_rfoot     0.279971
neck_lfoot     0.286010
neck_rhand     0.565957
neck_lhand     0.562241
neck_tip       0.323338
anus_chest     0.744020
anus_rfoot     0.627957
anus_lfoot     0.611615
anus_rhand     0.426210
anus_lhand     0.410555
anus_tip       0.560513
chest_rfoot    0.500490
chest_lfoot    0.501775
chest_rhand    0.586647
chest_lhand    0.564098
chest_tip      0.421291
rfoot_lfoot    0.246747
rfoot_rhand    0.260187
rfoot_lhand    0.282862
rfoot_tip      0.385252
lfoot_rhand    0.301125
lfoot_lhand    0.264743
lfoot_tip      0.356357
rhand_lhand    0.352924
rhand_tip      0.258368
lhand_tip      0.288193
dtype: float64

### Apply row by row

In [14]:
rolling_corr.groupby(level=0).apply(lambda x: corr_flatten(x)).unstack()

,lfoot_lhand,lfoot_rhand,lhand_rhand,rfoot_lfoot,rfoot_lhand,rfoot_rhand
0.55,0.420771,0.259220,-0.230745,0.737912,0.384960,-0.323462
0.60,0.408157,0.254535,-0.244439,0.673841,0.269956,-0.304451
0.65,0.396234,0.245848,-0.260584,0.543175,0.250063,-0.262604
0.70,0.397881,0.235739,-0.260875,0.534893,0.253469,-0.278345
0.75,0.412008,0.234537,-0.246392,0.530713,0.264173,-0.286510
...,...,...,...,...,...,...
99.30,-0.162736,0.560084,-0.140074,0.115347,0.158081,-0.103764
99.35,-0.099045,0.561452,-0.062731,0.122395,0.234593,-0.093553
99.40,-0.016024,0.562232,0.038515,0.175089,0.196344,-0.033585
99.45,0.029806,0.600330,0.043031,0.274165,0.162219,-0.081409


In [15]:
def get_rolling_corr(data, window=20, center=True):
    rolling_corr = data.rolling(window, center=center).corr()
    return rolling_corr.groupby(level=0).apply(lambda x: corr_flatten(x)).unstack()

# Usage

In [16]:
get_rolling_corr(ava.velocity[['rfoot', 'lfoot', 'lhand', 'rhand']]).iplot(kind='heatmap', colorscale='-rdbu')